In [9]:
import os
import scipy.io
import pandas as pd
import numpy as np


# Directory containing .mat files
folder_path = 'BPF/Data/'

# List all files in the directory
files = os.listdir(folder_path)

# Filter out .mat files
mat_files = [file for file in files if file.endswith('.mat')]


In [10]:
# Load each .mat file
dfs = []


for file in mat_files:
    file_path = os.path.join(folder_path, file)
    mat_data = scipy.io.loadmat(file_path)
    df = pd.DataFrame(mat_data["tfaIn"])
    label_column = np.zeros(len(df))  # Initialize with zeros
    label_column[:5000] = 0  # First 5000 rows get value 0
    label_column[-5000:] = 0  # Last 5000 rows get value 0
    label_column[5000:-5000] = 1  # In between get value 1
    df['label'] = label_column
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)


In [11]:
df

,0,1,2,3,4,5,6,7,8,9,label
0,29.398160,5.761961,7.968489,26.938175,-2.186015,-1.934578,-1.530351,0.408127,-14.092072,1.220974,0.0
1,23.703285,4.253463,7.811738,26.191340,-3.256789,-3.146197,-1.630288,1.012860,-15.688435,0.262867,0.0
2,16.489386,2.568731,7.295618,24.602441,-4.223104,-4.232556,-1.667098,1.497403,-16.689586,-0.749488,0.0
3,8.109756,0.752088,6.437778,22.197819,-5.057587,-5.158698,-1.646729,1.828552,-17.073881,-1.807628,0.0
4,-1.010505,-1.146025,5.273122,19.035149,-5.736066,-5.894696,-1.576792,1.982818,-16.846426,-2.899210,0.0
...,...,...,...,...,...,...,...,...,...,...,...
442859,-5.127009,24.278185,-17.447836,-2.091012,-3.276975,0.142129,-2.916217,-2.267223,-2.783660,29.645733,0.0
442860,-2.980641,27.673270,-15.084055,-0.335333,-3.499048,1.177863,-8.521929,-2.771866,-2.084361,33.339126,0.0
442861,-0.843042,29.850844,-12.958786,1.215300,-3.744888,2.047056,-14.058573,-3.274509,-1.375184,35.274779,0.0
442862,1.110703,30.586085,-11.223723,2.428130,-4.078570,2.650519,-19.405493,-3.834365,-0.783776,35.286524,0.0


In [13]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


In [14]:
import tensorflow as tf

def split_features_target(data):
  # Assuming the label is in the last column
  features = data[:, :-1]  # All columns except the last
  target = data[:, -1]  # Last column
  features = torch.tensor(features)
  target = torch.tensor(target)
  return features, target

# Split features and target
X, y = split_features_target(df.to_numpy())
y = tf.expand_dims(y, axis=1)



In [15]:
print(X.shape , y.shape)

torch.Size([442864, 10]) (442864, 1)


In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape
input_shape = (10,)  # Assuming each sample has 10 features

# Define the CNN model
model = models.Sequential([
    layers.Reshape(input_shape + (1,), input_shape=input_shape),  # Reshape input to match (10, 1) for grayscale
    layers.Conv1D(32, kernel_size=3, activation='relu'),           # Convolutional layer with 32 filters
    layers.MaxPooling1D(pool_size=2),                              # Max pooling layer
    layers.Flatten(),                                              # Flatten layer
    layers.Dense(128, activation='relu'),                         # Dense layer with 128 neurons
    layers.Dropout(0.5),                                           # Dropout layer with dropout rate of 0.5
    layers.Dense(1, activation='sigmoid')                          # Output layer with 1 neuron and sigmoid activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your input and target data
X_np = np.array(X)
y_np = np.array(y)

# Split the data into training and validation sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train_split, y_train_split, epochs=20, batch_size=32, validation_data=(X_test_split, y_test_split))



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 10, 1)             0         
                                                                 
 conv1d (Conv1D)             (None, 8, 32)             128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 32)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                        

In [19]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score

# Predict probabilities for each class
y_pred_prob = model.predict(X_test_split)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate precision
precision = precision_score(y_test_split, y_pred)

# Calculate recall
recall = recall_score(y_test_split, y_pred)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_split, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_split, y_pred)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)


2768/2768 [==============================] - 2s 778us/step
Precision: 0.8221569240387621
Recall: 0.9592810352625359
Confusion Matrix:
 [[ 5807 14223]
 [ 2791 65752]]
Accuracy: 0.8079098596637801


In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape
input_shape = (10,)  # Assuming each sample has 10 features

# Define the neural network model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=input_shape),  # Dense layer with 64 neurons and ReLU activation
    layers.Dense(64, activation='relu'),                            # Dense layer with 32 neurons and ReLU activation
    layers.Dropout(0.1),                                             # Dropout layer with dropout rate of 0.5
    layers.Dense(1, activation='sigmoid')                            # Output layer with 1 neuron and sigmoid activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your input and target data
X_np = np.array(X)
y_np = np.array(y)

# Split the data into training and validation sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train_split, y_train_split, epochs=20, batch_size=32, validation_data=(X_test_split, y_test_split))



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               1408      
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9,729
Trainable params: 9,729
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
11072/11072 [==============================] - 11s 956us/step - loss: 0.4504 - accuracy: 0.7784 - val_loss: 0.4246 - val_accuracy: 0.7908
Epoch 2/20
11072/11072 [===========

In [21]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score

# Predict probabilities for each class
y_pred_prob = model.predict(X_test_split)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate precision
precision = precision_score(y_test_split, y_pred)

# Calculate recall
recall = recall_score(y_test_split, y_pred)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_split, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_split, y_pred)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)


2768/2768 [==============================] - 2s 773us/step
Precision: 0.8363731442273353
Recall: 0.9566987146754592
Confusion Matrix:
 [[ 7201 12829]
 [ 2968 65575]]
Accuracy: 0.8216499384688336


In [22]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the input shape
input_shape = (10, 1)  # Assuming each sample has 10 features and reshape to fit RNN input

# Define the RNN model
model = models.Sequential([
    layers.SimpleRNN(64, input_shape=input_shape, return_sequences=True),  # Simple RNN layer with 64 units
    layers.SimpleRNN(32),                                                 # Another Simple RNN layer with 32 units
    layers.Dropout(0.5),                                                   # Dropout layer with dropout rate of 0.5
    layers.Dense(1, activation='sigmoid')                                  # Output layer with 1 neuron and sigmoid activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Display the model summary
import numpy as np
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your input and target data
X_np = np.array(X)
y_np = np.array(y)

# Split the data into training and validation sets
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train_split, y_train_split, epochs=20, batch_size=32, validation_data=(X_test_split, y_test_split))



Epoch 1/20
11072/11072 [==============================] - 30s 3ms/step - loss: 0.4800 - accuracy: 0.7691 - val_loss: 0.4451 - val_accuracy: 0.7826
Epoch 2/20
11072/11072 [==============================] - 29s 3ms/step - loss: 0.4486 - accuracy: 0.7832 - val_loss: 0.4329 - val_accuracy: 0.7917
Epoch 3/20
11072/11072 [==============================] - 40s 4ms/step - loss: 0.4415 - accuracy: 0.7863 - val_loss: 0.4298 - val_accuracy: 0.7931
Epoch 4/20
11072/11072 [==============================] - 36s 3ms/step - loss: 0.4380 - accuracy: 0.7881 - val_loss: 0.4295 - val_accuracy: 0.7924
Epoch 5/20
11072/11072 [==============================] - 36s 3ms/step - loss: 0.4354 - accuracy: 0.7892 - val_loss: 0.4291 - val_accuracy: 0.7935
Epoch 6/20
11072/11072 [==============================] - 36s 3ms/step - loss: 0.4341 - accuracy: 0.7895 - val_loss: 0.4258 - val_accuracy: 0.7958
Epoch 7/20
11072/11072 [==============================] - 38s 3ms/step - loss: 0.4334 - accuracy: 0.7891 - val_loss: 0

In [23]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score

# Predict probabilities for each class
y_pred_prob = model.predict(X_test_split)

# Convert probabilities to class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate precision
precision = precision_score(y_test_split, y_pred)

# Calculate recall
recall = recall_score(y_test_split, y_pred)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_split, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test_split, y_pred)

# Print the results
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", conf_matrix)
print("Accuracy:", accuracy)


2768/2768 [==============================] - 4s 2ms/step
Precision: 0.8018145937620331
Recall: 0.9721488700523759
Confusion Matrix:
 [[ 3560 16470]
 [ 1909 66634]]
Accuracy: 0.7924988427624671
